In [ ]:
import pandas as pd
import numpy as np

# Load the Excel file (header starts at row 2)
file_path = "Dataset_PlantAdvisor.xlsx"
df = pd.read_excel(file_path, header=1)

# Show current columns
print("✅ Columns detected:", df.columns.tolist())

# Fill missing Latitude & Longitude
if 'Longitude' in df.columns and 'Latitude' in df.columns:
    df['Longitude'] = df['Longitude'].fillna(method='ffill').fillna(method='bfill')
    df['Latitude'] = df['Latitude'].fillna(method='ffill').fillna(method='bfill')
else:
    print("⚠️ Column names not found — please check manually")

# 🪴 Add new 'Tanaman' column with several plant types
# Example plant list — you can change or extend this
plant_types = ["Padi", "Jagung", "Kedelai", "Kopi", "Tebu", "Cabai", "Tomat"]

# Randomly assign one plant per row
np.random.seed(42)  # for reproducibility
df["Tanaman"] = np.random.choice(plant_types, size=len(df))

# Save updated dataset to Excel and CSV
excel_out = "Dataset_PlantAdvisor_Filled.xlsx"
csv_out = "Dataset_PlantAdvisor_Filled.csv"
df.to_excel(excel_out, index=False)
df.to_csv(csv_out, index=False)

print("✅ Dataset updated and saved successfully!")
print(f"📄 Excel: {excel_out}")
print(f"📊 CSV:   {csv_out}")

# Display a small preview
df.head()

In [ ]:
csv_path = "Dataset_PlantAdvisor_Filled.csv"
df = pd.read_csv(csv_path)
df = df.drop(columns=["Nama Daerah", "Evidence"])

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
print("Categorical columns:", cat_cols)
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # cast to str just in case
    encoders[col] = le

In [ ]:
df.to_csv("data.csv", index=False)

In [ ]:
import torch
from customhopular import load_and_preprocess_csv, Hopular
from trainer import create_data_loaders, HopularTrainer


# Load and preprocess your CSV
X_train, X_val, X_test, y_train, y_val, y_test, metadata = load_and_preprocess_csv(
    csv_path="data.csv",
    target_column='Tanaman',
    categorical_columns=None,
    test_size=0.2,
    min_class_samples=2
)

# Create data loaders
train_loader, val_loader, test_loader = create_data_loaders(
    X_train, X_val, X_test, y_train, y_val, y_test,
    metadata, batch_size=32  # Use original batch size
)

# Create model
model = Hopular(
    input_sizes=metadata['input_sizes'],
    target_discrete=metadata['target_discrete'],
    target_numeric=metadata['target_numeric'],
    feature_discrete=metadata['feature_discrete'],
    memory=metadata['memory'],
    feature_size=32,
    hidden_size=16,
    hidden_size_factor=1.0,
    num_heads=4,
    num_blocks=1,
    scaling_factor=1.0,
    input_dropout=0.2,
    lookup_dropout=0.2,
    output_dropout=0.2,
)

# Create trainer
trainer = HopularTrainer(
    model,
    learning_rate=1e-3,
    weight_decay=0.01,
    initial_feature_loss_weight=1.0,
    final_feature_loss_weight=0.0
)

print("Attempting to run a full training epoch...")

# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    
trainer.fit(train_loader, val_loader, metadata, n_epochs=100, patience=10)

# 5. Evaluate
test_metrics = trainer.evaluate(test_loader, metadata)
print(f"Test Loss: {test_metrics['loss']:.4f}")
if test_metrics['accuracy']:
    print(f"Test Accuracy: {test_metrics['accuracy']:.2%}")

Testing the exact training scenario that causes tensor error...
Loaded dataset: 16 samples, 8 features
Task: Classification with 5 classes
Class distribution: {np.int64(0): np.int64(1), np.int64(1): np.int64(5), np.int64(2): np.int64(2), np.int64(3): np.int64(4), np.int64(4): np.int64(4)}
Removed classes: [0]
After filtering: 15 samples
Using stratified split (min class size: 2)
Attempting to run a full training epoch...


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 386.11it/s]



Epoch 1/100
  Train - Loss: 2.0384, Feature Loss: 2.0384, Target Loss: 0.0000
  Train - Target Acc: 0.00%
  Val - Loss: 0.0000, Acc: 0.00%
  LR: 0.001000, Annealing: 1.000
  → Model saved!


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 198.69it/s]



Epoch 2/100
  Train - Loss: 1.6963, Feature Loss: 1.6963, Target Loss: 0.0000
  Train - Target Acc: 0.00%
  Val - Loss: 0.0000, Acc: 0.00%
  LR: 0.000999, Annealing: 1.000


...


Epoch 10/100
  Train - Loss: 1.9661, Feature Loss: 1.9661, Target Loss: 0.0000
  Train - Target Acc: 0.00%
  Val - Loss: 0.0000, Acc: 0.00%
  LR: 0.000976, Annealing: 0.980


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 486.92it/s]



Epoch 11/100
  Train - Loss: 3.1209, Feature Loss: 3.1209, Target Loss: 0.0000
  Train - Target Acc: 0.00%
  Val - Loss: 0.0000, Acc: 0.00%
  LR: 0.000970, Annealing: 0.976

Early stopping at epoch 11

Training complete! Best model loaded.


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 220.39it/s]

Test Loss: 0.0000


In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
import pprint
pprint.pprint(metadata)
print(model)


In [ ]:
import customhopular
print(dir(customhopular))